In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!unzip /content/drive/MyDrive/FYP/chest_xray_MobileNetV3_Smote>.zip -d /content/extracted_files

In [2]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

In [3]:
# Step 1: Set up paths for the dataset
dataset_dir = "/content/extracted_files/chest_xray_MobileNetV3_Smote"

# Define paths for training and testing
train_dir = os.path.join(dataset_dir, "/content/extracted_files/chest_xray_MobileNetV3_Smote/train")
test_dir = os.path.join(dataset_dir, "/content/extracted_files/chest_xray_MobileNetV3_Smote/test")


In [4]:
# Image dimensions and batch size
img_height, img_width = 224, 224  # MobileNetV3 recommended input size


In [5]:
# Step 1: Load Images Without Augmentation
def load_images_and_labels(directory):
    data = []
    labels = []
    class_names = sorted(os.listdir(directory))  # Ensure consistent label mapping
    for class_idx, class_name in enumerate(class_names):
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                try:
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
                    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalize
                    data.append(img_array)
                    labels.append(class_idx)
                except:
                    print(f"Error loading image {img_path}")
    return np.array(data), np.array(labels), class_names



In [6]:
# Load training and test data
X_train, y_train, class_names_train = load_images_and_labels(train_dir)
X_test, y_test, class_names_test = load_images_and_labels(test_dir)


Error loading image /content/extracted_files/chest_xray_MobileNetV3_Smote/train/PNEUMONIA/.DS_Store


In [7]:
# Verify class mappings
print("Class Names (Training):", class_names_train)
print("Class Names (Testing):", class_names_test)


Class Names (Training): ['NORMAL', 'PNEUMONIA']
Class Names (Testing): ['NORMAL', 'PNEUMONIA']


In [8]:
# Step 2: Apply SMOTE to Balance Classes
# Flatten image data for SMOTE
X_train_flat = X_train.reshape((X_train.shape[0], -1))


In [ ]:
# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_flat, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [ ]:
# Reshape back to image format
X_train_resampled = X_train_resampled.reshape((-1, img_height, img_width, 3))


In [ ]:
# Verify new class distribution
unique, counts = np.unique(y_train_resampled, return_counts=True)
print("Class Distribution After SMOTE:", dict(zip(unique, counts)))
